In [33]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sci
import ciw

In [34]:
import ambulance_game as abg

In [35]:
def get_mean_blocking_time(pi, states, lambda_a, lambda_o=None, mu=None):
    mean_ambulances = abg.markov.get_mean_ambulances_blocked(pi, states)
    mean_blocking_time = (mean_ambulances / lambda_a)
    return mean_blocking_time

In [40]:
# Parameters
seed_num = None     
runtime = 14400
warm_up_time = 100
num_of_trials = 50
output_type = "tuple"
system_capacity = 60
parking_capacity = 40

In [41]:
def compare_simulation_markov(lambda_a, lambda_o, mu, num_of_servers, threshold, seed_num, runtime, warm_up_time, num_of_trials, output_type, system_capacity, parking_capacity):
    res = abg.simulation.get_multiple_runs_results(lambda_a=lambda_a, lambda_o=lambda_o, mu=mu, num_of_servers=num_of_servers, threshold=threshold, seed_num=seed_num, warm_up_time=warm_up_time, num_of_trials=num_of_trials, output_type=output_type, runtime=runtime, system_capacity=system_capacity, parking_capacity=parking_capacity)
    sim_mean_block = np.mean([np.mean(b.blocking_times) for b in res])

    all_states = abg.markov.build_states(threshold, system_capacity, parking_capacity)
    Q = abg.markov.get_transition_matrix(lambda_a, lambda_o, mu, num_of_servers, threshold, system_capacity, parking_capacity)
    pi = abg.markov.get_steady_state_algebraically(Q, algebraic_function=np.linalg.lstsq)

    markov_mean_block = get_mean_blocking_time(pi, all_states, lambda_a, mu=mu)
    
    return sim_mean_block, markov_mean_block

# Comparing blocking times with Little's law

In [42]:
# par = [lambda_a, lambda_o, mu, num_of_servers, threshold]
par_1 = [0.15, 0.2, 0.1, 8, 5]
par_2 = [0.15, 0.1, 0.1, 7, 4]
par_3 = [0.2, 0.3, 0.1, 12, 10]
par_4 = [0.2, 0.3, 0.1, 13, 5]

In [43]:
res1 = compare_simulation_markov(par_1[0], par_1[1], par_1[2], par_1[3], par_1[4], seed_num, runtime, warm_up_time, num_of_trials, output_type, system_capacity, parking_capacity)
res2 = compare_simulation_markov(par_2[0], par_2[1], par_2[2], par_2[3], par_2[4], seed_num, runtime, warm_up_time, num_of_trials, output_type, system_capacity, parking_capacity)
res3 = compare_simulation_markov(par_3[0], par_3[1], par_3[2], par_3[3], par_3[4], seed_num, runtime, warm_up_time, num_of_trials, output_type, system_capacity, parking_capacity)
res4 = compare_simulation_markov(par_4[0], par_4[1], par_4[2], par_4[3], par_4[4], seed_num, runtime, warm_up_time, num_of_trials, output_type, system_capacity, parking_capacity)

In [48]:
res1

(2.1280729831639538, 2.2074715188497813)

In [49]:
res2

(1.987419826598748, 1.9635149940302197)

In [50]:
res3

(0.0878194886998425, 0.08329506080883302)

In [51]:
res4

(14.848726478887867, 15.019252473424162)

# Compare waiting times without Little's Law

In [64]:
lambda_a = 0.1
lambda_o = 0.1
mu = 0.1
num_of_servers = 3
threshold = 100

In [75]:
# Parameters
seed_num = None     
runtime = 1440
warm_up_time = 100
num_of_trials = 100
output_type = "tuple"
system_capacity = 20
parking_capacity = 10

In [76]:
def get_mean_waiting_time(pi, states, lambda_a, lambda_o, mu, num_of_servers):
    states_np = np.array(states)
    indices = [index for index, (u,v) in enumerate(all_states) if v>num_of_servers and u==0]
    mean_waiting_time = np.sum(((states_np[indices][:,1] - num_of_servers) / (mu * num_of_servers)) * pi[indices])
    return mean_waiting_time

In [77]:
res = abg.simulation.get_multiple_runs_results(lambda_a=lambda_a, lambda_o=lambda_o, mu=mu, num_of_servers=num_of_servers, threshold=threshold, seed_num=seed_num, warm_up_time=warm_up_time, num_of_trials=num_of_trials, output_type=output_type, runtime=runtime, system_capacity=system_capacity, parking_capacity=parking_capacity)
sim_mean_wait = np.mean([np.mean(w.waiting_times) for w in res])

all_states = abg.markov.build_states(threshold, system_capacity, parking_capacity)
Q = abg.markov.get_transition_matrix(lambda_a, lambda_o, mu, num_of_servers, threshold, system_capacity, parking_capacity)
pi = abg.markov.get_steady_state_algebraically(Q, algebraic_function=np.linalg.lstsq)

markov_mean_wait = get_mean_waiting_time(pi, all_states, lambda_a, lambda_o, mu, num_of_servers)

In [78]:
sim_mean_wait

4.412087409496544

In [79]:
markov_mean_wait

2.935718022099208

# Break-down of formula for mean waiting time

In [14]:
states_np = np.array(all_states)

In [15]:
indices = [index for index, (u,v) in enumerate(all_states) if v>num_of_servers and u==0]

In [17]:
indices

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [18]:
states_np[indices][:,1]

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

In [19]:
states_np[indices][:,1] - num_of_servers

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [20]:
(states_np[indices][:,1] - num_of_servers) / (mu * num_of_servers)

array([ 3.33333333,  6.66666667, 10.        , 13.33333333, 16.66666667,
       20.        , 23.33333333, 26.66666667, 30.        , 33.33333333,
       36.66666667, 40.        , 43.33333333, 46.66666667, 50.        ,
       53.33333333])

In [21]:
((states_np[indices][:,1] - num_of_servers) / (mu * num_of_servers)) * pi[indices]

array([0.32936668, 0.43915558, 0.43915558, 0.39036051, 0.32530043,
       0.26024034, 0.20240915, 0.1542165 , 0.11566237, 0.08567583,
       0.06282894, 0.04569378, 0.03300106, 0.02369307, 0.01692362,
       0.01203458])

In [22]:
np.sum(((states_np[indices][:,1] - num_of_servers) / (mu * num_of_servers)) * pi[indices])

2.935718022099208